In [1]:
pip install gym==0.17.1 numpy==1.19.5


     |████████████████████████████████| 1.6 MB 7.8 MB/s 
  Created wheel for gym: filename=gym-0.17.1-py3-none-any.whl size=1648708 sha256=cf29eeffa4e5ab7554321c313f2334eb6f21428e1d511ab794c42e6feec12d24
  Stored in directory: /root/.cache/pip/wheels/12/7a/2a/2e85bca5dd2c3b319675a5db8a48837b7cfe0603240442b771
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3


In [2]:
import gym
import random
import numpy as np
import time
from collections import deque
import pickle
from collections import defaultdict

EPISODES =   20000
LEARNING_RATE = .1
DISCOUNT_FACTOR = .99
EPSILON = 1
EPSILON_DECAY = .999

def default_Q_value():
    return 0

if __name__ == "__main__":
    random.seed(1)
    np.random.seed(1)
    env = gym.envs.make("FrozenLake-v0")
    env.seed(1)
    env.action_space.np_random.seed(1)
    # Q_table = defaultdict(default_Q_value) # starts with a pessimistic estimate of zero reward for each state. #TODO
    episode_reward_record = deque(maxlen=100)

    action_size = env.action_space.n
    state_size = env.observation_space.n
    Q_table = np.zeros((state_size, action_size))

    for i in range(EPISODES):

        episode_reward = 0
        state = env.reset()
        done = False
        
        while done != True:
            tradeoff = random.uniform(0, 1)
            if tradeoff < EPSILON:
                action = env.action_space.sample()                
            else:
                action = np.argmax(Q_table[state,:])

            new_state, reward, done, info = env.step(action)
            # Q_table[state][action] = Q_table[state, action] + LEARNING_RATE*(reward+DISCOUNT_FACTOR*np.max(Q_table[new_state,:])- Q_table[state, action])

            #SARSA
            tradeoff = random.uniform(0, 1)        
            if tradeoff < EPSILON:
                new_action = env.action_space.sample()                
            else:
                new_action = np.argmax(Q_table[new_state,:])

            Q_table[state][action] = Q_table[state, action] + LEARNING_RATE*(reward+DISCOUNT_FACTOR*Q_table[new_state,new_action]- Q_table[state, action])
            episode_reward += reward
            state = new_state

        episode_reward_record.append(episode_reward)
        # EPSILON = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-EPSILON_DECAY*i) # TODO ... how to update epsilon
        EPSILON = EPSILON * EPSILON_DECAY
        if i%100 ==0 and i>0:
            print("LAST 100 EPISODE AVERAGE REWARD: " + str(sum(list(episode_reward_record))/100))
            print("EPSILON: " + str(EPSILON) )
    
    #what does 0 in Q-table mean?
    
    ####DO NOT MODIFY######
    print(Q_table)
    model_file = open('Q_TABLE.pkl' ,'wb')
    pickle.dump([Q_table,EPSILON],model_file)
    #######################


LAST 100 EPISODE AVERAGE REWARD: 0.0
EPSILON: 0.9038873549665959
LAST 100 EPISODE AVERAGE REWARD: 0.02
EPSILON: 0.8178301806491574
LAST 100 EPISODE AVERAGE REWARD: 0.01
EPSILON: 0.7399663251239436
LAST 100 EPISODE AVERAGE REWARD: 0.04
EPSILON: 0.6695157201007336
LAST 100 EPISODE AVERAGE REWARD: 0.02
EPSILON: 0.6057725659163237
LAST 100 EPISODE AVERAGE REWARD: 0.01
EPSILON: 0.548098260578011
LAST 100 EPISODE AVERAGE REWARD: 0.04
EPSILON: 0.4959150020176678
LAST 100 EPISODE AVERAGE REWARD: 0.05
EPSILON: 0.44869999946146477
LAST 100 EPISODE AVERAGE REWARD: 0.04
EPSILON: 0.4059802359226587
LAST 100 EPISODE AVERAGE REWARD: 0.09
EPSILON: 0.36732772934619257
LAST 100 EPISODE AVERAGE REWARD: 0.12
EPSILON: 0.33235524492954527
LAST 100 EPISODE AVERAGE REWARD: 0.11
EPSILON: 0.3007124156643058
LAST 100 EPISODE AVERAGE REWARD: 0.12
EPSILON: 0.2720822322326576
LAST 100 EPISODE AVERAGE REWARD: 0.07
EPSILON: 0.2461778670932771
LAST 100 EPISODE AVERAGE REWARD: 0.14
EPSILON: 0.22273980093919937
LAST 100

In [ ]:
def evaluate_frozen_lake(Q_table, EPSILON, visualize=False):
    total_reward = 0
    random.seed(1)
    np.random.seed(1)
    env = gym.envs.make("FrozenLake-v0")
    env.seed(1)
    env.action_space.np_random.seed(1)


    for i in range(100):
        obs = env.reset()
        done = False
        while done == False:
            if random.uniform(0,1) < EPSILON:
                action = env.action_space.sample()
            else:
                prediction = np.array([Q_table[(obs,i)] for i in range(env.action_space.n)])
                action =  np.argmax(prediction)
            obs,reward,done,info = env.step(action)
            total_reward += reward
            if visualize:
                env.render()
                time.sleep(.01)
    score = total_reward/100
    return score

def test_Q_learning(visualize = False):
    # loaded_data = pickle.load(open('Q_TABLE.pkl', 'rb'))
    # Q_table = loaded_data[0]
    # EPSILON = loaded_data[1]
    score = evaluate_frozen_lake(Q_table,EPSILON,visualize = False)
    points = 0
    if score >= 0.6:
        points = 30
    elif score >= 0.4:
        points = 15 
    print("Q Learning on FrozenLake-v0:")
    print("Average episode-reward over 100 episodes is " + str(score))
    print(str(points)  +" out of 30 points received.")
    return points

test_Q_learning()

Q Learning on FrozenLake-v0:
Average episode-reward over 100 episodes is 0.65
30 out of 30 points received.


30